# NFL DFS Lineup Optimizer

In [1]:
# Store filenames as variables
week = 9
qbFile = "QBWeek9.csv"
rbFile = "RBWeek9.csv"
wrFile = "WRWeek9.csv"
teFile = "TEWeek9.csv"
dFile = "DefWeek9.csv"
dkFile = "DKSalariesWeek9.csv"


## Read in DraftKings player pool and salaries

In [2]:
import csv
import pandas as pd
import re

def load_DK(filename):
    salaries = pd.read_csv(filename)
    salaries = salaries[["Position", "Name", "ID", "Salary", "TeamAbbrev", "Game Info"]]
    
    # Gets opponent and stores in new variable
    game = salaries["Game Info"]
    gameSpl = game.str.split(" ") # splits each string by space
    teams = [] # list of games "CAR@TB"
    for g in range(len(gameSpl)): # loops over each split string and stores first element (game)
        teams.append((gameSpl[g][0]))
        
    df = pd.Series(teams) # convert to series
    df = df.str.split("@") # split teams by '@'
    away = []
    home = []
    for a in range(len(df)):
        away.append((df[a][0]))
        home.append((df[a][1]))
    
    # Add away and home team to "salaries"
    salaries = salaries.assign(Away = away, Home = home)
    salaries.drop(['Game Info'], axis=1, inplace=True) 
    
    #pd.set_option("display.max_rows", None, "display.max_columns", None)
    return(salaries)

## Read in FantasyPros projections

In [3]:
def load_projections(QB, RB, WR, TE, Def):
    # Reads .csv for each position's FP projections (PPR, max/min/median projections)
    QB = pd.read_csv(QB)
    RB = pd.read_csv(RB)
    WR = pd.read_csv(WR)
    TE = pd.read_csv(TE)
    Def = pd.read_csv(Def)
    
    # Removes 2nd row and drops unnecessary columns 
    QB = QB.drop(0)[["Player", "Team", "FPTS"]]
    RB = RB.drop(0)[["Player", "Team", "FPTS"]]
    WR = WR.drop(0)[["Player", "Team", "FPTS"]]
    TE = TE.drop(0)[["Player", "Team", "FPTS"]]
    Def = Def[["Player", "Team", "FPTS"]]
    
    # Adds "Position" column
    QB["Position"] = "QB"
    RB["Position"] = "RB"
    WR["Position"] = "WR"
    TE["Position"] = "TE"
    Def["Position"] = "D/ST"
 
    # Combines all positions into 1 DF
    all_pos = pd.concat([QB, RB, WR, TE, Def], axis=0, ignore_index = True)
    
    # placeholder vectors for floor and ceiling projected points by player
    floor = all_pos["FPTS"][2::3] # gets every 3rd value starting at 3rd row (low)
    ceiling = all_pos["FPTS"][1::3] # gets every 3rd value starting at 2nd row
   
    # gets rid of messy 2nd and 3rd rows for each player (includes every 3rd row, starting with row 1)
    all_pos = all_pos[::3]
    
    # adds floor and ceiling vectors into all_pos DF
    all_pos = all_pos.set_index(floor.index)
    all_pos["Floor"] = floor
    all_pos = all_pos.set_index(ceiling.index)
    all_pos["Ceiling"] = ceiling
    
    # rename "FPTS" to "Median" and Re-order columns
    all_pos = all_pos.rename(columns={"FPTS" : "Median"})
    all_pos = all_pos[["Player", "Team", "Position", "Floor", "Median", "Ceiling"]]
    
    # Reset index of data frame
    all_pos = all_pos.reset_index(drop=True)
    
    # display all players
    pd.set_option("display.max_rows", None, "display.max_columns", None)
    return(all_pos)

## Name matching between sites

The following code creates a function that takes in 2 string vectors (DK Salary names and FP Projections names) and matches them. This finds any tiny differences between the sites naming conventions. For example, Will Fuller and Will Fuller V are the same person. I want to assign an ID to each name so that in the future I can match on ID. 

In [4]:
from fuzzywuzzy import fuzz # import fuzzy string matching library

names_DK = load_DK(dkFile)["Name"]
names_FP = load_projections(qbFile, rbFile, wrFile, teFile, dFile)["Player"]

def name_match(vector1, vector2, ratio=89):
    index = 0
    d = ()
    ell = []
    for a in vector1: 
        for b in vector2:
            if fuzz.ratio(a.lower(), b.lower()) > ratio:
                d = (index, a, b) 
                index += 1
                ell.append((d))
    
    # convert to data frame
    df = pd.DataFrame(ell, columns=["Index", "DK Name", "FP Name"])
    return(df)
    
df_Names = name_match(names_DK, names_FP)

C:\Users\punco\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## Merge Salary and FP Projection data on this new Index

In [5]:
# Add indexes to both data frames
# adding to salary data
new = load_DK(dkFile)
salary_DF = pd.merge(new, df_Names, left_on="Name" , right_on="DK Name")
salary_DF.drop(['DK Name', 'FP Name'], axis=1, inplace=True)

# adding to FP data frame
new2 = load_projections(qbFile, rbFile, wrFile, teFile, dFile)
FP_DF = pd.merge(new2, df_Names, left_on="Player", right_on="FP Name")
FP_DF.drop(['DK Name', 'FP Name'], axis=1, inplace=True)

# Merge salary data with projection data on "Index"
sal_proj = pd.merge(FP_DF, salary_DF, on = ["Index", "Position"])
sal_proj = sal_proj.drop_duplicates(subset=['ID']) # Removes all those Chris Thompson's

# Getting rid of duplicate columns, keeping Draftkings columns "Name" and "TeamAbbrev"
sal_proj.drop(['Player', 'Team'], axis=1, inplace=True)

# Drop low projected players
sal_proj = sal_proj[sal_proj['Ceiling'] >= 5]
print(sal_proj)

    Position  Floor  Median  Ceiling  Index                   Name        ID  \
0         QB   21.2    23.7     29.7      3        Patrick Mahomes  15713425   
1         QB   20.1    23.2     29.6      7         Russell Wilson  15713427   
2         QB   21.6    23.2     28.9      6           Kyler Murray  15713426   
3         QB   20.4    22.2     25.6     12         Deshaun Watson  15713428   
4         QB   20.8    22.1     25.8     14             Josh Allen  15713429   
5         QB   20.9    22.0     28.7     20         Justin Herbert  15713431   
6         QB   18.8    20.0     26.9     17          Lamar Jackson  15713430   
7         QB   16.7    18.4     22.2     30     Ben Roethlisberger  15713433   
8         QB   17.9    18.4     22.0     31              Matt Ryan  15713434   
9         QB   16.6    18.0     23.2     26       Matthew Stafford  15713432   
10        QB   17.2    17.8     23.0     46             Derek Carr  15713439   
11        QB   16.0    17.6     21.7    

In [6]:
def df_stuff(df):
    df = df[df["Salary"] > 7000]
    print(df)

df_stuff(sal_proj)

    Position  Floor  Median  Ceiling  Index                 Name        ID  \
0         QB   21.2    23.7     29.7      3      Patrick Mahomes  15713425   
1         QB   20.1    23.2     29.6      7       Russell Wilson  15713427   
2         QB   21.6    23.2     28.9      6         Kyler Murray  15713426   
3         QB   20.4    22.2     25.6     12       Deshaun Watson  15713428   
47        RB   21.1    22.1     24.3      0  Christian McCaffrey  15713483   
48        RB   15.6    20.6     23.4      2          Dalvin Cook  15713485   
49        RB   17.3    18.7     20.8      4        Derrick Henry  15713487   
139       WR   18.4    20.3     23.9      1      DeAndre Hopkins  15713695   
141       WR   17.6    19.2     22.0     10          Julio Jones  15713701   
142       WR   16.8    18.8     21.9      8         Stefon Diggs  15713699   
146       WR   13.1    17.0     21.7      5           DK Metcalf  15713697   
147       WR   14.2    17.0     19.1     11          Tyreek Hill

## print N largest projections

In [7]:
def Nmaxelements(list1, N): 
    points = [] 
    list1 = list(list1)
    for i in range(0, N):  
        max1 = 0
        for p in list1:
            if p > max1:
                max1 = p
    
        list1.remove(max1)
        points.append(max1)
        
    return points
    
Nmaxelements(sal_proj["Ceiling"], 1)       

[29.7]

## Add position constraints
I want the 9 highest scores, but now there has to be exactly 1 QB, 2 RB, 3 WR, 1 TE, 1 flex, and 1 Def

In [8]:
def Nmaxelements_pos(df):
    points = []
    total_players = 0
    total_QB = 0
    total_WR = 0
    total_RB = 0
    total_TE = 0
    total_Def = 0
    salary = 0
    qbs = df[df["Position"] == "QB"]
    rbs = df[df["Position"] == "RB"]
    wrs = df[df["Position"] == "WR"]
    te = df[df["Position"] == "TE"]
    dst = df[df["Position"] == "D/ST"]
    
    max1 = 0
    while total_QB < 1:
        for p in qbs["Ceiling"]:
            if p > max1:
                max1 = p
                total_QB += 1
    
                
                
        while total_RB < 3:
            Nmaxelements(rbs["Ceiling"], 3)
            total_RB += 1 
        print(df.loc[df['Ceiling'] == Nmaxelements(rbs["Ceiling"], 3)[0], "Name"].iloc[0])
        print(df.loc[df['Ceiling'] == Nmaxelements(rbs["Ceiling"], 3)[1], "Name"].iloc[0])
        print(df.loc[df['Ceiling'] == Nmaxelements(rbs["Ceiling"], 3)[2], "Name"].iloc[0])
    
    print(qbs.loc[qbs['Ceiling'] == max1, "Name"].iloc[0])
                            
Nmaxelements_pos(sal_proj)

Christian McCaffrey
Dalvin Cook
Derrick Henry
Patrick Mahomes


# Defense data

In [9]:
projections = load_projections(qbFile, rbFile, wrFile, teFile, dFile)
salaries = load_DK(dkFile)
a = salaries[salaries.Position=="DST"]["Name"]
b = projections[projections.Position=="D/ST"]["Player"]

# Get nickname from projections
blist = []
for i in b:
    c = i.strip() # remove whitespace
    c = c.split()
    blist.append(c[-1])

# add nickname to projections
proj = projections.loc[projections.Position=="D/ST"].copy()
proj["Nickname"] = blist


# data frame: salaries for D/ST
salariesD = salaries.loc[salaries.Position=="DST"].copy()

# Remove whitespace from salaries' names
alist = []
for i in a:
    stripped = i.strip()
    alist.append(stripped)

# Add name back to salaries
salariesD["StripName"] = alist


# Change WAS Football Team
proj.loc[proj["Nickname"] == 'Team', "Nickname"] = "Washington"
salariesD.loc[salariesD["StripName"] == "WAS Football Team", "StripName"] = "Washington"

# Merge on StripName and Nickname (salariesD, proj)
sal_proj_D = pd.merge(salariesD, proj, how = "left", left_on="StripName", right_on="Nickname")

# Add index column
x = range(1000, len(sal_proj_D)+1000)
sal_proj_D["Index"] = x

# Remove unnecessary columns and re-order
sal_proj_D.drop(["Position_x", "StripName", "Player", "Team", "Nickname"], axis=1, inplace=True)
sal_proj_D = sal_proj_D[["Position_y", "Floor", "Median", "Ceiling", "Index", "Name", "ID", "Salary", "TeamAbbrev",
                       "Away", "Home"]]
sal_proj_D.rename(columns={'Position_y':'Position'}, inplace=True)

print(sal_proj_D)


   Position  Floor  Median  Ceiling  Index                Name        ID  \
0      D/ST    6.9     8.9     10.6   1000           Steelers   15714181   
1      D/ST    5.7     6.2      7.7   1001             Chiefs   15714182   
2      D/ST    5.3     6.2      7.0   1002             Ravens   15714183   
3      D/ST    6.4     7.3      9.2   1003  WAS Football Team   15714184   
4      D/ST    6.4     6.5      7.4   1004              Colts   15714185   
5      D/ST    4.0     6.1      9.8   1005              Bears   15714186   
6      D/ST    6.0     6.5      7.8   1006             Texans   15714187   
7      D/ST    5.2     5.4      6.2   1007           Chargers   15714188   
8      D/ST    6.2     5.8      7.2   1008             Titans   15714189   
9      D/ST    6.3     6.3      6.9   1009          Cardinals   15714190   
10     D/ST    5.3     6.1      6.7   1010           Seahawks   15714191   
11     D/ST    4.4     6.2      7.4   1011           Dolphins   15714192   
12     D/ST 

## Run program

In [10]:
# Load DK Salaries and Fantasy Pros projections
salaries = load_DK(dkFile)
projections = load_projections(qbFile, rbFile, wrFile, teFile, dFile)

# Store player names from DK and FP in separate vectors
names_DK = salaries["Name"]
names_FP = projections["Player"]

# Run name match function to assign index to players
df_Names = name_match(names_DK, names_FP)

# Merge names with salary data frame
salaries = pd.merge(salaries, df_Names, left_on="Name" , right_on="DK Name")
salaries.drop(['DK Name', 'FP Name'], axis=1, inplace=True)

# Merge names with projections data frame
projections = pd.merge(projections, df_Names, left_on="Player", right_on="FP Name")
projections.drop(['DK Name', 'FP Name'], axis=1, inplace=True)

# Merge salaries and projections on the new "index" value
sal_proj = pd.merge(projections, salaries, on = ["Index", "Position"])
sal_proj = sal_proj.drop_duplicates(subset=['ID']) # Removes all those Chris Thompson's

# Getting rid of duplicate columns, keeping Draftkings columns "Name" and "TeamAbbrev"
sal_proj.drop(['Player', 'Team'], axis=1, inplace=True)

# Add defense to end of data frame
sal_proj  = sal_proj.append(sal_proj_D)

# Drop low projected players
sal_proj = sal_proj[sal_proj['Median'] >= 5]

print(sal_proj.head(5))

  Position  Floor  Median  Ceiling  Index             Name        ID  Salary  \
0       QB   21.2    23.7     29.7      3  Patrick Mahomes  15713425    8100   
1       QB   20.1    23.2     29.6      7   Russell Wilson  15713427    7600   
2       QB   21.6    23.2     28.9      6     Kyler Murray  15713426    7800   
3       QB   20.4    22.2     25.6     12   Deshaun Watson  15713428    7100   
4       QB   20.8    22.1     25.8     14       Josh Allen  15713429    7000   

  TeamAbbrev Away Home  
0         KC  CAR   KC  
1        SEA  SEA  BUF  
2        ARI  MIA  ARI  
3        HOU  HOU  JAX  
4        BUF  SEA  BUF  


# Write to .csv

In [11]:
sal_proj.to_csv(r'C:\Users\punco\OneDrive\Desktop\Fantasy Football\Week 9.csv', index = False)

From here, I can use excel solver to produce 1 optimal lineup with Draft Kings constraints, or I could write a program in Python that produces the n most optimal lineups.

## Experimenting with stacking (matching QB with 1 or 2 pass catchers or RB, bring back with 1 or 2 players from other team)

for every qb
    find numToPair players from same team
    return dataframe: "QB" | QBPoints | QBSalary | "Player1" | Player1Points | Player1Salary | ... TotalSalary | TotalPts

In [12]:
for q in sal_proj[sal_proj.Position=="QB"].itertuples():
    print(q)

Pandas(Index=0, Position='QB', Floor=21.2, Median=23.7, Ceiling=29.7, _5=3, Name='Patrick Mahomes', ID=15713425, Salary=8100, TeamAbbrev='KC', Away='CAR', Home='KC')
Pandas(Index=1, Position='QB', Floor=20.1, Median=23.2, Ceiling=29.6, _5=7, Name='Russell Wilson', ID=15713427, Salary=7600, TeamAbbrev='SEA', Away='SEA', Home='BUF')
Pandas(Index=2, Position='QB', Floor=21.6, Median=23.2, Ceiling=28.9, _5=6, Name='Kyler Murray', ID=15713426, Salary=7800, TeamAbbrev='ARI', Away='MIA', Home='ARI')
Pandas(Index=3, Position='QB', Floor=20.4, Median=22.2, Ceiling=25.6, _5=12, Name='Deshaun Watson', ID=15713428, Salary=7100, TeamAbbrev='HOU', Away='HOU', Home='JAX')
Pandas(Index=4, Position='QB', Floor=20.8, Median=22.1, Ceiling=25.8, _5=14, Name='Josh Allen', ID=15713429, Salary=7000, TeamAbbrev='BUF', Away='SEA', Home='BUF')
Pandas(Index=5, Position='QB', Floor=20.9, Median=22.0, Ceiling=28.7, _5=20, Name='Justin Herbert', ID=15713431, Salary=6800, TeamAbbrev='LAC', Away='LV', Home='LAC')
Pan

In [13]:
def stack_DF(df, rb=False):
    qb_df = df[df.Position=="QB"] # QB only data frame with QB only
    crit1 = df["Position"] !="QB"
    crit2 = df["Position"] !="D/ST"
    crit3 = df["Position"] !="RB"
    if(rb)==True:
        others_df = df[crit1 & crit2] # data frame with RB, WR, TE only
    else:
        others_df = df[crit1 & crit2 & crit3] # Data frame with WR, TE only
    
    playerList = []
    
    for q in qb_df.itertuples(): # iterate over qbs
        playerList.append(q) # add to new list
        for o in others_df.itertuples(): # iterate over non-qbs
            if(q[9]==o[9]): # add to new list if on same team as qb
                playerList.append(o)
    
    playerDF = pd.DataFrame(playerList)
    return(playerDF)

stackedDF = stack_DF(sal_proj)
print(stackedDF)

     Index Position  Floor  Median  Ceiling   _5                   Name  \
0        0       QB   21.2    23.7     29.7    3        Patrick Mahomes   
1      147       WR   14.2    17.0     19.1   11            Tyreek Hill   
2      182       WR    7.8     9.2     10.4   82         Mecole Hardman   
3      194       WR    4.5     6.4      7.4  209      Demarcus Robinson   
4      252       TE   15.1    17.2     19.7    9           Travis Kelce   
5        1       QB   20.1    23.2     29.6    7         Russell Wilson   
6      143       WR   16.5    18.3     21.0   21          Tyler Lockett   
7      146       WR   13.1    17.0     21.7    5             DK Metcalf   
8      191       WR    5.4     7.0      8.6  218            David Moore   
9      267       TE    6.0     6.5      7.5  219             Greg Olsen   
10       2       QB   21.6    23.2     28.9    6           Kyler Murray   
11     139       WR   18.4    20.3     23.9    1        DeAndre Hopkins   
12     161       WR    8.